In [727]:
from importlib import reload

import parallelsim

from parallelsim import Work, ComputeAndWeightWorkers


num_workers = 6
num_stages = 6
num_batches = 6

In [ ]:
def DistributedDataParallel(work: Work):
    return ComputeAndWeightWorkers(work.batch, work.batch)


assert num_batches == num_workers


parallelsim.simulate(num_workers, num_stages, num_batches, DistributedDataParallel)

In [725]:
def FullyShardedDistributedDataParallel(work: Work):
    return ComputeAndWeightWorkers(work.batch, work.stage % num_workers)


assert num_batches == num_workers


parallelsim.simulate(
    num_workers, num_stages, num_batches, FullyShardedDistributedDataParallel
)

In [ ]:
def GPipe(work: Work):
    return ComputeAndWeightWorkers(work.stage, work.stage)


assert num_stages == num_workers

parallelsim.simulate(num_workers, num_stages, num_batches, GPipe)

In [756]:
assert num_stages == num_workers

parallelsim.simulate(
    num_workers,
    num_stages,
    num_batches,
    GPipe,
    parallelsim.OldestBatchFirst(num_stages, num_batches),
)

In [757]:
group_size = 3


def LoopedPipelineParallelism(work: Work):
    def h(x, y):
        return (group_size * y % num_workers) + (x % group_size)

    worker = h(work.stage, work.batch)
    return ComputeAndWeightWorkers(worker, worker)


parallelsim.simulate(
    num_workers,
    num_stages,
    num_batches,
    LoopedPipelineParallelism,
    parallelsim.OldestBatchFirst(num_stages, num_batches),
)

In [542]:
group_size = 3


def FullyShardedLoopedPipelineParallelism(work: Work):
    def h(x, y):
        return (group_size * y % num_workers) + (x % group_size)

    return ComputeAndWeightWorkers(h(work.stage, work.batch), h(work.stage, work.stage))


parallelsim.simulate(
    num_workers, num_stages, num_batches, FullyShardedLoopedPipelineParallelism
)